<a href="https://colab.research.google.com/github/Erickrus/leetcode/blob/master/variable_injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Variable Injection**
Variable injection is a skill to change the value of any running cell. So during the long running job. you can tune or change the value as you wish.

In [121]:
## Infrastructure
import threading
import queue
import time

class GracefullyStop:
  def __init__(self, stopping=False):
    self.stopping = stopping


class RunnableStdout(threading.Thread):
  def __init__(self, stdout, gracefullyStop, *args, **kwargs):
    super().__init__(*args, **kwargs)  
    self.stdout = stdout
    self.gracefullyStop = gracefullyStop

  def run(self):
    while True:
      if self.gracefullyStop.stopping:
        break
      time.sleep(0.01)
      try:
        line=self.stdout.get(timeout=10.)
        print(line)
      except:
        pass
    print("stdout stopped")

class Runnable(threading.Thread):
  def __init__(self, callable, param={},*args, **kwargs):
    super().__init__(*args, **kwargs)
    self.callable = callable
    self.param = param
    self.gracefullyStop = GracefullyStop()
    self.stdout = queue.Queue()
    self.runnableStdout = RunnableStdout(self.stdout, self.gracefullyStop)
  def run(self):
    self.runnableStdout.start()
    self.callable(self.gracefullyStop, self.param, self.stdout)
    #self.runnableStdout.join()
    print("Runnable stopped")
    #time.sleep(1)

  def stop(self):
    self.gracefullyStop.stopping = True



In [122]:
import time
a = 0
def a_piece_of_code(gracefullyStop, param, stdout):
  global a 
  for i in range(40):
    a +=1
    stdout.put(str(a))
    time.sleep(3)
    if gracefullyStop.stopping:
      break
  print("a_piece_of_code stopped")
  time.sleep(1)

In [123]:
r = Runnable(a_piece_of_code)
r.start()

In [132]:
r.stop()